# Finetuning Whisper-large-V2 on Colab using PEFT-Lora training

In this Colab, we present a step-by-step guide on how to fine-tune Whisper for any multilingual ASR dataset using Hugging Face 🤗 Transformers and 🤗 PEFT. 

For more details on model, datasets and metrics, refer blog [Fine-Tune Whisper For Multilingual ASR with 🤗 Transformers](https://huggingface.co/blog/fine-tune-whisper)



## Inital Setup

In [5]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

/bin/bash: add-apt-repository: command not found
/bin/bash: apt: command not found
/bin/bash: apt: command not found


In [1]:
import os
os.environ['HF_HOME'] = '/scratch/s5521904/thesis/cache/'

In [2]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install -q bitsandbytes datasets accelerate
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-gcbi_i8r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-gcbi_i8r
  Resolved https://github.com/huggingface/transformers to commit 0d84901cb7e797c90653e2c8ca2ce2a6b3498208
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 22.5 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9019458 sha256=f214e376d2f85438602ce358fd94228d51e35b50e92c46ee537c281c1f06a5a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-gbap4qbd/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully

Linking the notebook to the Hub is straightforward - it simply requires entering your Hub authentication token when prompted. Find your Hub authentication token [here](https://huggingface.co/settings/tokens):

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
# Select CUDA device index
import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['HF_HOME'] = '/scratch/s5521904/thesis/cache/'
model_name_or_path = "openai/whisper-small"
language = "frisian"
language_abbr = "fy-NL"
task = "transcribe"



In [10]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Mon Jun 10 22:36:36 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          On  |   00000000:E3:00.0 Off |                    0 |
| N/A   39C    P0             58W /  250W |   11724MiB /  40960MiB |     55%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## Load Dataset

In [4]:
from datasets import load_dataset, load_metric, Audio
dataset = load_dataset("csv", data_files={"train": ['/scratch/s5521904/thesis/cv-corpus-6.1-2020-12-11/fy-NL/train_large.csv'], 
                                          "test": '/scratch/s5521904/thesis/cv-corpus-6.1-2020-12-11/fy-NL/test.csv'})

In [5]:
dataset = dataset.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['path', 'sentence'],
        num_rows: 7475
    })
    test: Dataset({
        features: ['path', 'sentence'],
        num_rows: 3020
    })
})


## Prepare Feature Extractor, Tokenizer and Data

In [6]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

In [7]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language= 'dutch', task=task)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name_or_path, language='dutch', task=task)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Prepare Data

In [10]:
print(dataset["train"][0])
print(dataset["test"][0])

{'path': '/scratch/s5521904/thesis/cv-corpus-6.1-2020-12-11/fy-NL/clips/common_voice_fy-NL_23796740.mp3', 'sentence': '"Hy hat te lang omrûn mei dat probleem."'}
{'path': '/scratch/s5521904/thesis/cv-corpus-6.1-2020-12-11/fy-NL/clips/common_voice_fy-NL_23761003.mp3', 'sentence': '"Ha jo faaks in gerjocht sûnder fleis of fisk foar my?"'}


Since
our input audio is sampled at 48kHz, we need to _downsample_ it to
16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model.

We'll set the audio inputs to the correct sampling rate using dataset's
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place,
but rather signals to `datasets` to resample audio samples _on the fly_ the
first time that they are loaded:

In [9]:
dataset = dataset.cast_column("path", Audio(sampling_rate=16000))

In [12]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['path', 'sentence'],
        num_rows: 7475
    })
    test: Dataset({
        features: ['path', 'sentence'],
        num_rows: 3020
    })
})


Re-loading the first audio sample in the Common Voice dataset will resample
it to the desired sampling rate:

In [14]:
print(dataset["train"][1])

{'path': {'path': '/scratch/s5521904/thesis/cv-corpus-6.1-2020-12-11/fy-NL/clips/common_voice_fy-NL_23802689.mp3', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        4.65313292e-07, -3.88197236e-07, -1.60365312e-06]), 'sampling_rate': 16000}, 'sentence': '"De fraach is dan ek wat soe Skylge wêze sonder rekreanten."'}


Now we can write a function to prepare our data ready for the model:
1. We load and resample the audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We encode the transcriptions to label ids through the use of the tokenizer.

In [10]:
from whisper_normalizer.basic import BasicTextNormalizer

normalizer = BasicTextNormalizer()

def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["path"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    
    normalized_text = normalizer(batch["sentence"])

    # encode target text to label ids
    batch["labels"] = tokenizer(normalized_text).input_ids
    return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method. The argument `num_proc` specifies how many CPU cores to use. Setting `num_proc` > 1 will enable multiprocessing. If the `.map` method hangs with multiprocessing, set `num_proc=1` and process the dataset sequentially.

In [11]:
dataset = dataset.map(prepare_dataset,remove_columns=dataset.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/7475 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/3020 [00:00<?, ? examples/s]

In [14]:
dataset["train"]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 7475
})

## Training and Evaluation

### Define a Data Collator

In [12]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [13]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metrics

We'll use the word error rate (WER) metric, the 'de-facto' metric for assessing
ASR systems. For more information, refer to the WER [docs](https://huggingface.co/metrics/wer). We'll load the WER metric from 🤗 Evaluate:

In [14]:
import evaluate

metric = evaluate.load("wer")

We then simply have to define a function that takes our model
predictions and returns the WER metric. This function, called
`compute_metrics`, first replaces `-100` with the `pad_token_id`
in the `label_ids` (undoing the step we applied in the
data collator to ignore padded tokens correctly in the loss).
It then decodes the predicted and label ids to strings. Finally,
it computes the WER between the predictions and reference labels:

In [15]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    pred_str_clean = [normalizer(pred_str) for pred in pred_str]
    label_str_clean = [normalizer(label_str) for label in label_str]

    wer = 100 * metric.compute(predictions=pred_str_clean, references=label_str_clean)

    return {"wer": wer}

### Load a Pre-Trained Checkpoint

Now let's load the pre-trained Whisper `small` checkpoint. Again, this
is trivial through use of 🤗 Transformers!

In [16]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, device_map="auto")

# model.hf_device_map - this should be {" ": 0}

Override generation arguments - no tokens are forced as decoder outputs (see [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)), no tokens are suppressed during generation (see [`suppress_tokens`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.suppress_tokens)):

In [17]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

### Post-processing on the model

Finally, we need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast all non `int8` layers in `float32` for stability.

In [18]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

### Apply LoRA

Here comes the magic with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.

In [19]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 245,273,856 || trainable%: 1.442854145857274


In [23]:
#from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

#config = LoraConfig(r=16, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

#model = get_peft_model(model, config)
#model.print_trainable_parameters()

trainable params: 1,880,064 || all params: 243,614,976 || trainable%: 0.771735806586866


In [24]:
#from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

#config = LoraConfig(r=2, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

#model = get_peft_model(model, config)
#model.print_trainable_parameters()

trainable params: 331,776 || all params: 242,066,688 || trainable%: 0.13705975107157248


In [29]:
#from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

#config = LoraConfig(r=2, lora_alpha=64, target_modules=["q_proj", "v_proj", "o_proj", "k_proj"], lora_dropout=0.05, bias="none")

#model = get_peft_model(model, config)
#model.print_trainable_parameters()

trainable params: 331,776 || all params: 242,066,688 || trainable%: 0.13705975107157248


In [25]:
#from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

#config = LoraConfig(r=64, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

#model = get_peft_model(model, config)
#model.print_trainable_parameters()

trainable params: 7,188,480 || all params: 248,923,392 || trainable%: 2.887828235925694


In [33]:
#from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

#config = LoraConfig(r=8, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

#model = get_peft_model(model, config)
#model.print_trainable_parameters()

trainable params: 1,327,104 || all params: 243,062,016 || trainable%: 0.545993990274482


In [28]:
#from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

#config = LoraConfig(r=128, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

#model = get_peft_model(model, config)
#model.print_trainable_parameters()

trainable params: 14,155,776 || all params: 255,890,688 || trainable%: 5.531962147837127


We are ONLY using **1%** of the total trainable parameters, thereby performing **Parameter-Efficient Fine-Tuning**

### Define the Training Configuration

In the final step, we define all the parameters related to training. For more detail on the training arguments, refer to the Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [23]:

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="Frisian_32r_LoRA_10h",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3,
    warmup_steps=50,
    num_train_epochs=15,
    evaluation_strategy="epoch",
    fp16=True,
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=25,
    report_to=["tensorboard"],
    #load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
)

**Few Important Notes:**
1. `remove_unused_columns=False` and `label_names=["labels"]` are required as the PeftModel's forward doesn't have the signature of the base model's forward.

2. INT8 training required autocasting. `predict_with_generate` can't be passed to Trainer because it internally calls transformer's `generate` without autocasting leading to errors.

3. Because of point 2, `compute_metrics` shouldn't be passed to `Seq2SeqTrainer` as seen below. (commented out)

In [21]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR


class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [25]:

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.524300,0.630007
2,0.432000,0.560740
3,0.352500,0.519197
4,0.271900,0.496120
5,0.225100,0.477897
6,0.176100,0.450545
7,0.134400,0.443545
8,0.101900,0.441382
9,0.071700,0.419683
10,0.043400,0.414869


TrainOutput(global_step=14025, training_loss=0.176752733363506, metrics={'train_runtime': 15168.2875, 'train_samples_per_second': 7.392, 'train_steps_per_second': 0.925, 'total_flos': 3.292903590912e+19, 'train_loss': 0.176752733363506, 'epoch': 15.0})

In [38]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_6_1",
    "dataset": "mozilla-foundation/common_voice_6_1",  # a 'pretty' name for the training dataset
    "dataset_args": "config: frisian, split: validation",
    #"language": "frisian",
    "model_name": "LoRA-Frisian-10h-new",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [39]:
trainer.push_to_hub(**kwargs)

adapter_model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

events.out.tfevents.1717247517.a100gpu5.2730885.3:   0%|          | 0.00/46.7k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/xuliu15/Frisian_32r_LoRA_10h_new/commit/1911f1af7977ab28c2927129e9776459a814dd94', commit_message='End of training', commit_description='', oid='1911f1af7977ab28c2927129e9776459a814dd94', pr_url=None, pr_revision=None, pr_num=None)

# Evaluation and Inference

**Important points to note while inferencing**:
1. As `predict_with_generate` can't be used, we will write the eval loop with `torch.cuda.amp.autocast()` as shown below.
2. As the base model is frozen, PEFT model sometimes fails ot recognise the language while decoding.Hence, we force the starting tokens to mention the language we are transcribing. This is done via `forced_decoder_ids = processor.get_decoder_prompt_ids(language="Marathi", task="transcribe")` and passing that to the `model.generate` call.
3. Please note that [AutoEvaluate Leaderboard](https://huggingface.co/spaces/autoevaluate/leaderboards?dataset=mozilla-foundation%2Fcommon_voice_11_0&only_verified=0&task=automatic-speech-recognition&config=mr&split=test&metric=wer) for `mr` language on `common_voice_11_0` has a bug wherein openai's `BasicTextNormalizer` normalizer is used while evaluation leading to degerated output text, an example is shown below:
```
without normalizer: 'स्विच्चान नरुवित्तीची पद्दत मोठ्या प्रमाणात आमलात आणल्या बसोन या दुपन्याने अनेक राथ प्रवेश केला आहे.'
with normalizer: 'स व च च न नर व त त च पद दत म ठ य प रम ण त आमल त आणल य बस न य द पन य न अन क र थ प रव श क ल आह'
```
Post fixing this bug, we report the 2 metrics for the top model of the leaderboard and the PEFT model:
1. `wer`: `wer` without using the `BasicTextNormalizer` as it doesn't cater to most indic languages. This is want we consider as true performance metric.
2. `normalized_wer`: `wer` using the `BasicTextNormalizer` to be comparable to the leaderboard metrics.
Below are the results:

| Model          | DrishtiSharma/whisper-large-v2-marathi | smangrul/openai-whisper-large-v2-LORA-colab |
|----------------|----------------------------------------|---------------------------------------------|
| wer            | 35.6457                                | 36.1356                                     |
| normalized_wer | 13.6440                                | 14.0165                                     |

We see that PEFT model's performance is comparable to the fully fine-tuned model on the top of the leaderboard. At the same time, we are able to train the large model in Colab notebook with limited GPU memory and the added advantage of resulting checkpoint being jsut `63` MB.



In [18]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_model_id = "xuliu15/Frisian_32r_LoRA_10h"

peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, device_map="auto"
)
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

In [20]:
#with data prepare normalization -new
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc
#from transformers import Seq2SeqGenerationConfig
eval_dataloader = DataLoader(dataset["test"], batch_size=1, collate_fn=data_collator)

model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    decoder_input_ids=batch["labels"][:, :4].to("cuda"),
                    max_new_tokens=255,
                    language = "English"
                    
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
#             decoded_preds = normalizer(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            
            
            
            normalized_preds = [normalizer(pred) for pred in decoded_preds]
            #normalized_labels = [normalizer(label) for label in decoded_labels]
            print("prediction:", normalized_preds)
            print("truth:", decoded_labels)
            metric.add_batch(
                
                predictions=normalized_preds,
                references=decoded_labels,
            )
    del generated_tokens, labels, batch
    gc.collect()
wer = 100 * metric.compute()

print(f"{wer=}")

  0%|          | 1/3020 [00:00<33:59,  1.48it/s]

prediction: [' ha jo faaks in gerjocht sûnder fleist of fisk foar my ']
truth: [' ha jo faaks in gerjocht sûnder fleis of fisk foar my ']


  0%|          | 2/3020 [00:01<28:51,  1.74it/s]

prediction: [' it nachtlibben hat net folle om e hakken ']
truth: [' it nachtlibben hat net folle om e hakken ']


  0%|          | 3/3020 [00:01<29:53,  1.68it/s]

...

prediction: [' app erfloed beskiede it deistich libben op e waadeilannen ']
truth: [' eb en floed beskiede it deistich libben op de waadeilannen ']


100%|█████████▉| 3019/3020 [28:17<00:00,  1.76it/s]

prediction: [' sûnt de bekendmakking ôfrûne woastje is der by in soad minske wat betiising ']
truth: [' sûnt de bekendmakking ôfrûne woansdei is der by in soad minsken wat betizing ']


100%|██████████| 3020/3020 [28:17<00:00,  1.78it/s]

prediction: [' de waafersissings binne geunstich de wyn jout him wat del ']
truth: [' de waarfoarsizzings binne geunstich de wyn jout him wat del ']
wer=23.503505619561004
